In [47]:
#import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [48]:
pd.set_option('display.max_columns', None)

In [49]:
# Assuming 'data' is a dictionary of DataFrames, replace '...' with the actual data
dataframes = {
    '2014': pd.read_csv('/content/drive/MyDrive/Data Science 2024/qualifying dataset/Team Qualifier Dataset and Document/ghs-2014-person-v1.csv'),
    '2015': pd.read_csv('/content/drive/MyDrive/Data Science 2024/qualifying dataset/Team Qualifier Dataset and Document/ghs-2015-person-v1.csv')
}
# Define the columns to keep initially

initial_fields = [
    'Age', 'age_grp', 'gender', 'race', 'prov', 'geotype', 'Metro',
    'Q26aASM', 'Q26aDBT', 'Q26aCAN', 'Q26aHIV', 'Q26aHBP', 'Q26aRTH', 'Q26aOTH',
    'disab', 'sevdisab', 'undisab',
    'Q15HIEDU', 'Q110ATTE',
    'Q124aABS', 'Q124bDAY', 'Q124cRSN',
    'Q41aWGE', 'Q41bBUS', 'Q41cVOL', 'Q41dRET', 'Q42aSTO', 'q42msal', 'Q45SEC',
    'Q46aLOOK', 'Q46cRNSW',
    'Q44aTM', 'Q44bMIN',
    'Q31aSG', 'Q31bOAG', 'Q31bDIS', 'Q31bCSG', 'Q31bCAR', 'Q31bFOS', 'Q31bWVT', 'Q31bGRN', 'Q31bSOC',
    'q11RELSH', 'q12aMARST', 'Q12bSPOU'
]

# Load the data from the DataFrame
df = dataframes['2014'][initial_fields]

# Combine chronic condition columns into one "Chronic Condition" column
chronic_columns = ['Q26aASM', 'Q26aDBT', 'Q26aCAN', 'Q26aHIV', 'Q26aHBP', 'Q26aRTH', 'Q26aOTH']
df['Chronic Condition'] = df[chronic_columns].apply(lambda x: any(x == 'Yes'), axis=1)

# Function to categorize education levels
def categorize_education(level):
    if pd.isna(level):
        return 'Unknown'
    level = str(level)  # Ensure level is string to prevent errors
    if 'No schooling' in level:
        return 'No Schooling'
    elif any(f'Grade {str(digit)}' in level for digit in range(1, 6)):
        return 'Primary Education'
    elif any(f'Grade {str(digit)}' in level for digit in range(6, 8)):
        return 'Lower Secondary Education'
    elif any(f'Grade {str(digit)}' in level for digit in range(8, 10)):
        return 'Upper Secondary Education'
    elif any(f'Grade {str(digit)}' in level for digit in range(10, 12)):
        return 'Further Secondary Education'
    elif 'Grade 12' in level or 'Matric' in level:
        return 'Matric'
    elif 'Diploma' in level:
        return 'Diploma'
    elif 'Certificate' in level:
        return 'Certificate'
    elif 'Bachelors' in level:
        return 'Bachelors'
    elif 'Honours' in level or 'Higher Diploma' in level:
        return 'Honours'
    elif 'Masters' in level or 'Doctorate' in level:
        return 'Postgraduate'
    else:
        return 'Other'

df['Education Level'] = df['Q15HIEDU'].apply(categorize_education)

# Combine commute details
df['Commute'] = df.apply(lambda x: f"{x['Q44aTM']} - {x['Q44bMIN']} mins", axis=1)

# Combine social grants into one comprehensive field
social_grant_cols = ['Q31aSG', 'Q31bOAG', 'Q31bDIS', 'Q31bCSG', 'Q31bCAR', 'Q31bFOS', 'Q31bWVT', 'Q31bGRN', 'Q31bSOC']
df['Social Grants'] = df[social_grant_cols].apply(lambda x: ', '.join(filter(pd.notna, x.astype(str))), axis=1)

# Define function to categorize employment status
def categorize_employment(wage, bus, vol, ret):
    if wage == 'Yes':
        return 'Employed'
    elif bus == 'Yes':
        return 'Self-employed'
    elif vol == 'Yes':
        return 'Volunteering'
    elif ret == 'Yes':
        return 'Returning to work'
    else:
        return 'Unemployed'

# Apply the function to categorize employment status
df['Employment Status'] = df.apply(lambda x: categorize_employment(x['Q41aWGE'], x['Q41bBUS'], x['Q41cVOL'], x['Q41dRET']), axis=1)

# Select demographic fields to keep
demographic_fields = ['Age', 'gender', 'race', 'prov', 'geotype', 'Employment Status']

# Drop fields that have been processed or are no longer needed
fields_to_drop = list(set(initial_fields) - set(demographic_fields))
df.drop(fields_to_drop, axis=1, inplace=True)

def normalize_commute(commute):
    if pd.isna(commute):
        return 'Not Applicable', 'Not Applicable'

    # Split the commute information into mode and time
    parts = commute.split(' - ')
    mode = parts[0].strip()
    time = parts[1].strip() if len(parts) > 1 else 'Unspecified'

    # Normalize transportation modes
    mode_mapping = {
        'Own car/other private vehicle/company vehicle': 'Private Vehicle',
        'Minibus taxi/sedan taxi/bakkie taxi': 'Taxi',
        'Bicycle/motorcycle': 'Bicycle/Motorcycle',
        'Office is at home': 'Home Office',
        'Not applicable': 'Not Applicable',
        'Other (specify)': 'Other',
        'Unspecified': 'Unspecified'
    }
    normalized_mode = mode_mapping.get(mode, 'Other')

    # Normalize commute times
    time_mapping = {
        'Less than 15 minutes': '<15 minutes',
        '15–30 minutes': '15-30 minutes',
        '31–60 minutes': '31-60 minutes',
        '61–90 minutes': '61-90 minutes',
        'More than 90 minutes': '>90 minutes',
        'Not applicable': 'Not Applicable',
        'Unspecified': 'Unspecified',
        'Do not know': 'Unknown'
    }
    normalized_time = time_mapping.get(time, 'Unknown')

    return normalized_mode, normalized_time

# Apply normalization function to the Commute column
df['Transportation Mode'], df['Commute Time'] = zip(*df['Commute'].apply(normalize_commute))

def normalize_social_grants(grants):
    # Split the grants string into individual responses
    responses = grants.split(', ')
    # Check if 'Yes' is in the responses to determine if any grant is received
    if 'Yes' in responses:
        return 'Yes'
    else:
        return 'No'

# Apply normalization function to the Social Grants column
df['Social Grants'] = df['Social Grants'].apply(normalize_social_grants)

# Optionally, drop the original Commute column if no longer needed
df.drop('Commute', axis=1, inplace=True)
df.drop('Commute Time', axis=1, inplace=True)
df.head()

<ipython-input-49-3087d26f391d>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chronic Condition'] = df[chronic_columns].apply(lambda x: any(x == 'Yes'), axis=1)
<ipython-input-49-3087d26f391d>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Education Level'] = df['Q15HIEDU'].apply(categorize_education)
<ipython-input-49-3087d26f391d>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,Age,gender,race,prov,geotype,Chronic Condition,Education Level,Social Grants,Employment Status,Transportation Mode
0,33,1,4,1,5,False,Other,No,Unemployed,Other
1,29,2,4,1,5,False,Other,No,Unemployed,Other
2,1,1,4,1,5,False,Other,No,Unemployed,Other
3,46,1,2,1,5,False,Other,No,Unemployed,Other
4,49,2,2,1,5,False,Other,No,Unemployed,Other


In [50]:
dataframes['2014'].head()

,uqnr,personnr,psu,prov,B_NIGHTS,gender,Age,age_grp,race,q11RELSH,q12aMARST,Q12bSPOU,Q12CPSNN,Q13aFATH,Q13bFalive,Q13cFPAR,Q13dFPSN,Q14aMOTH,Q14bMalive,Q14cMPAR,Q14dMPSN,Q15HIEDU,Q16ATECHILD,Q17DA,Q18ECD,Q19WRTN,Q19READ,Q19FORM,Q19WRTL,Q19CAL,Q19RDSN,Q110ATTE,Q111RSNN,Q112EDUI,Q113PPRIV,Q114DLRN,Q115aMOD,Q115bTIM,Q115cNR,Q115dNNI,Q116FEET,Q117RNFE,Q118BURS,Q119ALanguage,Q119Amathematics,Q119BTEXTBOOKS,Q119cBOOK,Q119cPOOR,Q119cLACK,Q119cBADC,Q119cFEES,Q119cCLAS,Q119cTABS,Q119cPMAN,Q119cOTHR,Q120GRDE,Q121SAME,Q122aFOOD,Q122bEFOOD,Q123aVIOL,Q123bVIOL1,Q123bVIOL2,Q123bVIOL3,Q123bVIOL4,Q123bVIOL5,Q123bVIOL6,Q124aABS,Q124bDAY,Q124cRSN,Q21MEDI,Q22GENHEALTH,Q23FLU,Q23DIA,Q23SCblood,Q23SUB,Q23DEP,Q23SEX,Q23PNEU,Q23BRON,Q23EPIL,Q24aCON,Q24bYNT,Q25aMVHoccupant,Q25aMVHpedestrian,Q25aBCL,Q25aGUN,Q25aSTR,Q25aCrime,Q25aFIRE,Q25aACCP,Q25aINTP,Q25aSPORTS,Q25aOTH,Q25bmiss,Q26aASM,Q26aDBT,Q26aCAN,Q26aHIV,Q26aHBP,Q26aRTH,Q26aSTRK,Q26aHEART,Q26aTB,Q26aMental,Q26aEPIL,Q26aMEN,Q26aPNEU,Q26aBRON,Q26aCHOLE,Q26aOSTER,Q26aOTH,Q26bASM,Q26bDBT,Q26bCAN,Q26bHIV,Q26bHBP,Q26bRTH,Q26bSTRK,Q26bHEART,Q26bTB,Q26bMental,Q26bEPIL,Q26bMEN,Q26bPNEU,Q26bBRON,Q26bCHOLE,Q26bOSTER,Q26bOTH,Q27aPRE,Q27bSTA,Q28SEE,Q28HEA,Q28WAL,Q28REM,Q28SEL,Q28COM,Q29EYE,Q29HAID,Q29WST,Q29WCH,Q29CMED,Q29OTH,Q31aSG,Q31bOAG,Q31bDIS,Q31bCSG,Q31bCAR,Q31bFOS,Q31bWVT,Q31bGRN,Q31bSOC,Q32DISG,Q41aWGE,Q41bBUS,Q41cVOL,Q41dRET,Q42aSTO,Q42bSP,Q43SALC,Q44aTM,Q44bMIN,Q45SEC,Q46aLOOK,Q46bSTART,Q46cRNSW,Q46dAccJOB,Q47JCR,Q48CELL,Q49RELIGIOUS,Q410ATEREL,Q411RESP,undisab,disab,sevdisab,literacy3b,geotype,q42msal,Metro,employ_Status1,employ_Status2,stratum,rotation,person_wgt
0,101000170000001501,1,10100017000,1,1,1,33,7,4,1,2,1,2,1,8,2,88,1,8,9,99,12,8,8,8,8,8,8,8,8,8,2,6,88,8,8,88,8,8,88,88,8,8,8,8,8,8,8,8,8,8,8,8,8,8,88,8,8,8,8,8,8,8,8,8,8,8,8,88,1,3,2,2,2,2,2,2,2,2,2,8,88,2,2,2,2,2,2,2,2,2,2,2,8,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,8,8,1,1,1,1,1,1,2,2,2,2,1,2,2,8,8,8,8,8,8,8,8,8,1,2,2,9,8000,2,99,2,1,1,9,9,99,9,2,1,1,4,2,0,0,0,0,5,8000,2,1,1,101501,4,512.302826
1,101000170000001501,2,10100017000,1,1,2,29,6,4,2,1,1,1,1,8,2,88,1,8,9,99,23,8,8,8,8,8,8,8,8,8,2,8,88,8,8,88,8,8,88,88,8,8,8,8,8,8,8,8,8,8,8,8,8,8,88,8,8,8,8,8,8,8,8,8,8,8,8,88,1,3,2,2,2,2,2,2,2,2,2,8,88,2,2,2,2,2,2,2,2,2,2,2,8,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,2,8,1,1,1,1,1,1,2,2,2,2,2,2,2,8,8,8,8,8,8,8,8,8,2,1,2,9,5000,2,99,1,8,2,9,9,99,9,2,1,1,4,2,0,0,0,0,5,5000,2,1,1,101501,4,512.302826
2,101000170000001501,3,10100017000,1,1,1,1,1,4,3,7,8,88,1,8,1,1,1,8,1,2,98,6,1,1,8,8,8,8,8,8,8,88,88,8,8,88,8,8,88,88,8,8,8,8,8,8,8,8,8,8,8,8,8,8,88,8,8,8,8,8,8,8,8,8,8,8,8,88,1,3,2,2,2,2,2,2,2,2,2,8,88,2,2,2,2,2,2,2,2,2,2,2,8,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,2,8,8,8,8,8,8,8,8,8,8,8,8,8,888888888,8,88,88,8,8,8,8,88,8,8,2,99,4,2,0,0,0,0,5,888888888,2,8,8,101501,4,512.302826
3,101000170000005701,1,10100017000,1,1,1,46,10,2,1,2,1,2,2,2,8,88,2,2,8,88,10,8,8,8,8,8,8,8,8,8,2,6,88,8,8,88,8,8,88,88,8,8,8,8,8,8,8,8,8,8,8,8,8,8,88,8,8,8,8,8,8,8,8,8,8,8,8,88,2,5,2,2,2,2,2,2,2,2,2,8,88,2,2,2,2,2,2,2,2,2,2,2,8,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,1,1,1,1,1,1,2,2,2,2,2,2,2,8,8,8,8,8,8,8,8,8,1,2,2,9,450,1,99,2,1,1,9,9,99,9,2,2,1,2,2,0,0,0,0,5,1800,2,1,1,101501,4,304.050736
4,101000170000005701,2,10100017000,1,1,2,49,10,2,2,2,1,1,2,2,8,88,2,2,8,88,1,8,8,8,1,1,2,2,1,1,2,10,88,8,8,88,8,8,88,88,8,8,8,8,8,8,8,8,8,8,8,8,8,8,88,8,8,8,8,8,8,8,8,8,8,8,8,88,2,3,2,2,2,2,2,2,2,2,2,8,88,2,2,2,2,2,2,2,2,2,2,2,8,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,2,8,1,1,1,1,1,1,2,2,2,2,2,2,2,8,8,8,8,8,8,8,8,8,2,2,2,2,888888888,8,88,88,8,8,2,2,88,8,2,2,1,2,2,0,0,0,0,5,888888888,2,9,3,101501,4,304.050736


In [51]:
# Define the columns to keep initially
initial_fields = [
    'Age', 'age_grp1', 'gender', 'race', 'prov', 'GeoType', 'Metro',
    'Q26aASM', 'Q26aDBT', 'Q26aCAN', 'Q26aHIV', 'Q26aHBP', 'Q26aRTH', 'Q26aSTRK', 'Q26aHEART', 'Q26aTB', 'Q26aOTH',
    'disab', 'Sevdisab', 'undisab', 'Q15HIEDU', 'Q110ATTE',
    'Q124aABS', 'Q124bDAY', 'Q124cRSN',
    'Q41awge', 'Q41bbus', 'Q41cVOL', 'Q41dret', 'Q42asto', 'Q42bsp', 'Q45sec', 'Q43salc',
    'Q46alook', 'Q46crnsw',
    'Q44atm', 'Q44bmin',
    'Q31aSG', 'Q31bOAG', 'Q31bDIS', 'Q31bCSG', 'Q31bCAR', 'Q31bFOS', 'Q31bWVT', 'Q31bGRN', 'Q31bSOC',
    'q11RELSH', 'q12aMARST', 'Q12bSPOU'
]

# Load the data from the DataFrame
df = dataframes['2015'][initial_fields]

# Combine chronic condition columns into one "Chronic Condition" column
chronic_columns = ['Q26aASM', 'Q26aDBT', 'Q26aCAN', 'Q26aHIV', 'Q26aHBP', 'Q26aRTH', 'Q26aSTRK', 'Q26aHEART', 'Q26aTB', 'Q26aOTH']
df['Chronic Condition'] = df[chronic_columns].apply(lambda x: any(x == 'Yes'), axis=1)

# Function to categorize education levels
def categorize_education(level):
    if pd.isna(level):
        return 'Unknown'
    level = str(level)  # Ensure level is string to prevent errors
    if 'No schooling' in level:
        return 'No Schooling'
    elif any(f'Grade {str(digit)}' in level for digit in range(1, 6)):
        return 'Primary Education'
    elif any(f'Grade {str(digit)}' in level for digit in range(6, 8)):
        return 'Lower Secondary Education'
    elif any(f'Grade {str(digit)}' in level for digit in range(8, 10)):
        return 'Upper Secondary Education'
    elif any(f'Grade {str(digit)}' in level for digit in range(10, 12)):
        return 'Further Secondary Education'
    elif 'Grade 12' in level or 'Matric' in level:
        return 'Matric'
    elif 'Diploma' in level:
        return 'Diploma'
    elif 'Certificate' in level:
        return 'Certificate'
    elif 'Bachelors' in level:
        return 'Bachelors'
    elif 'Honours' in level or 'Higher Diploma' in level:
        return 'Honours'
    elif 'Masters' in level or 'Doctorate' in level:
        return 'Postgraduate'
    else:
        return 'Other'

df['Education Level'] = df['Q15HIEDU'].apply(categorize_education)

# Combine commute details
df['Commute'] = df.apply(lambda x: f"{x['Q44atm']} - {x['Q44bmin']} mins", axis=1)

# Combine social grants into one comprehensive field
social_grant_cols = ['Q31aSG', 'Q31bOAG', 'Q31bDIS', 'Q31bCSG', 'Q31bCAR', 'Q31bFOS', 'Q31bWVT', 'Q31bGRN', 'Q31bSOC']

df['Social Grants'] = df[social_grant_cols].apply(lambda x: ', '.join(filter(pd.notna, x.astype(str))), axis=1)

# Define function to categorize employment status
def categorize_employment(wage, bus, vol, ret):
    if wage == 'Yes':
        return 'Employed'
    elif bus == 'Yes':
        return 'Self-employed'
    elif vol == 'Yes':
        return 'Volunteering'
    elif ret == 'Yes':
        return 'Returning to work'
    else:
        return 'Unemployed'

# Apply the function to categorize employment status
df['Employment Status'] = df.apply(lambda x: categorize_employment(x['Q41awge'], x['Q41bbus'], x['Q41cVOL'], x['Q41dret']), axis=1)

# Select demographic fields to keep
demographic_fields = ['Age', 'gender', 'race', 'prov', 'GeoType', 'Employment Status']

# Drop fields that have been processed or are no longer needed
fields_to_drop = list(set(initial_fields) - set(demographic_fields))
df.drop(fields_to_drop, axis=1, inplace=True)

def normalize_commute(commute):
    if pd.isna(commute):
        return 'Not Applicable', 'Not Applicable'

    # Split the commute information into mode and time
    parts = commute.split(' - ')
    mode = parts[0].strip()
    time = parts[1].strip() if len(parts) > 1 else 'Unspecified'

    # Normalize transportation modes
    mode_mapping = {
        'Own car/other private vehicle/company vehicle': 'Private Vehicle',
        'Minibus taxi/sedan taxi/bakkie taxi': 'Taxi',
        'Bicycle/motorcycle': 'Bicycle/Motorcycle',
        'Office is at home': 'Home Office',
        'Not applicable': 'Not Applicable',
        'Other (specify)': 'Other',
        'Unspecified': 'Unspecified'
    }
    normalized_mode = mode_mapping.get(mode, 'Other')

    # Normalize commute times
    time_mapping = {
        'Less than 15 minutes': '<15 minutes',
        '15–30 minutes': '15-30 minutes',
        '31–60 minutes': '31-60 minutes',
        '61–90 minutes': '61-90 minutes',
        'More than 90 minutes': '>90 minutes',
        'Not applicable': 'Not Applicable',
        'Unspecified': 'Unspecified',
        'Do not know': 'Unknown'
    }
    normalized_time = time_mapping.get(time, 'Unknown')

    return normalized_mode, normalized_time

# Apply normalization function to the Commute column
df['Transportation Mode'], df['Commute Time'] = zip(*df['Commute'].apply(normalize_commute))

def normalize_social_grants(grants):
    # Split the grants string into individual responses
    responses = grants.split(', ')
    # Check if 'Yes' is in the responses to determine if any grant is received
    if 'Yes' in responses:
        return 'Yes'
    else:
        return 'No'

# Apply normalization function to the Social Grants column
df['Social Grants'] = df['Social Grants'].apply(normalize_social_grants)

# Optionally, drop the original Commute column if no longer needed
df.drop('Commute', axis=1, inplace=True)
df.drop('Commute Time', axis=1, inplace=True)
df.head()

<ipython-input-51-f60ec5e3eda3>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Chronic Condition'] = df[chronic_columns].apply(lambda x: any(x == 'Yes'), axis=1)
<ipython-input-51-f60ec5e3eda3>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Education Level'] = df['Q15HIEDU'].apply(categorize_education)
<ipython-input-51-f60ec5e3eda3>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,Age,gender,race,prov,GeoType,Chronic Condition,Education Level,Social Grants,Employment Status,Transportation Mode
0,48,1,2,1,3,False,Other,No,Unemployed,Other
1,43,1,2,1,3,False,Other,No,Unemployed,Other
2,42,2,2,1,3,False,Other,No,Unemployed,Other
3,54,1,2,1,3,False,Other,No,Unemployed,Other
4,36,2,2,1,3,False,Other,No,Unemployed,Other
